Pre Processing function

In [1]:
remove_words = ["@handle", "RT", "http"]

def pre_process(sentence, max_length):
    sentence = sentence.split()
    target_remove = set()
    for token in sentence:
        for target in remove_words:
            if (target == "http") and (target in token.lower()):
                target_remove.add(token)
                break
            if target in token:
                target_remove.add(token)
                break  
    for target in target_remove:
        while target in sentence:
            sentence.remove(target)
    max_length = max(max_length, len(sentence))
    sentence = ' '.join(sentence)
    return sentence, max_length


Load Train data to train_dict. 
train_dict[id] = [[train_instace1], [train_instance2] ....]

In [7]:
import csv
import collections
import numpy as np
import pandas as pd

train_file_path = "data/train_tweets.txt"
train_dict = collections.defaultdict(list)
max_length = 0

length_array = []
with open(train_file_path, encoding='utf-8') as tsvfile:
    reader = tsvfile.readlines()
    for i, row in enumerate(reader):
        row = row.strip().split("\t")
        id = int(row[0])
        instance, max_length = pre_process(row[1], max_length)
        if not instance == "":
            train_dict[id].append(instance)
            length_array.append(len(instance.split()))
    print("Total rows: %d" % i)
    
print("Total ids: %d" % len(train_dict))
print("Longest Sentence: %d" % (max_length))


Total rows: 328931
Total ids: 9295
Longest Sentence: 37


In [3]:
import pandas as pd
length_array = np.array(length_array)
sorted_len_array = sorted(length_array)
print("median: {0}".format(length_array))
s = pd.Series(length_array)
s.describe()

median: [ 8 11  6 ... 12 10 15]


count    327874.000000
mean         12.756400
std           6.679779
min           1.000000
25%           7.000000
50%          12.000000
75%          18.000000
max          37.000000
dtype: float64

In [4]:
import numpy as np

tweetsPerPersonMap = {}
for key in train_dict.keys():
    tweetsLen = len(train_dict[key])
    if tweetsLen not in tweetsPerPersonMap.keys():
        tweetsPerPersonMap[tweetsLen] = [key]
    else:
        tweetsPerPersonMap[tweetsLen].append(key)

sortedTweetsNumList = sorted(tweetsPerPersonMap.keys())
sortedTweetsSumList = []
userSumList = []
for tweetsNum in sortedTweetsNumList:
    userNum = len(tweetsPerPersonMap[tweetsNum])
    sortedTweetsSumList.append(tweetsNum*userNum)
    userSumList.append(userNum)

sortedTweetsSumList = np.cumsum(sortedTweetsSumList)
userSumList = np.cumsum(userSumList)
userSumPropotionList = userSumList/userSumList[-1]
sortedTweetsProportionList = sortedTweetsSumList/sortedTweetsSumList[-1]

In [5]:
# save tweets num per person
tweets_num_path = 'data/v1_1/tweets_num_per_person.txt'
with open(tweets_num_path, 'w') as txtFile:
    for index, tweetsNum in enumerate(sortedTweetsNumList):
        userList = tweetsPerPersonMap[tweetsNum]
        proportion = sortedTweetsProportionList[index]
        userProportion = userSumPropotionList[index]
        #txtFile.write("{0:.2f},{1:.2f},{2},{3}\n".format(proportion, userProportion, tweetsNum, userList))
        txtFile.write("{0:.2f},{1:.2f},{2}\n".format(proportion, userProportion, tweetsNum, userList))

In [6]:
# store new dict
TWEETS_NUM_THRESHOLD= 55

sortedTweetsNumList = np.array(sortedTweetsNumList)
newTweetsNumList = sortedTweetsNumList[sortedTweetsNumList>TWEETS_NUM_THRESHOLD]

print(len(newTweetsNumList))

newTrainDict = {}
for index, tweetsNum in enumerate(newTweetsNumList):
    for user in tweetsPerPersonMap[tweetsNum]:
        newTrainDict[user] = train_dict[user]

145


In [7]:
import random
dev_split = 0.1
train_split = 1 - dev_split

dev_set_dict = {}
train_set_dict = {}

for id in newTrainDict:
    target_list = newTrainDict[id]
    length = len(target_list)
    random.shuffle(target_list)
    split = int(np.ceil(length*dev_split))
    dev_set_dict[id] = target_list[:split]
    train_set_dict[id] = target_list[split:length]
#     print(len(dev_set_dict[id]), len(train_set_dict[id]), length)

print(len(dev_set_dict), len(train_set_dict), len(newTrainDict))

1132 1132 1132


In [8]:
import pickle
import pandas as pd

def save_to_file(target_dict, file_path):
    id_list = []
    sentence_list = []
    for id in target_dict:
        for sentence in target_dict[id]:
            id_list.append(id)
            sentence_list.append(sentence)
      
    id_list = np.array(id_list)
    sentence_list = np.array(sentence_list)
    random_index = np.array(range(len(sentence_list)))
    random.shuffle(random_index)
    id_list = id_list[random_index]
    sentence_list = sentence_list[random_index]
    
    with open(file_path,'w') as output_file:
        for i in range(len(id_list)):
            result = "{0}\t{1}\n".format(id_list[i], sentence_list[i])
            output_file.write(result)
    #dataframe = pd.DataFrame({'id':id_list,'sentence':sentence_list})
    #dataframe.to_csv(file_path,index=False,sep='\t',header=None)
    print(len(id_list))
    return

dev_set_path = 'data/v1_1/dev_set_v1_1_more_than_{0}.txt'.format(TWEETS_NUM_THRESHOLD)
train_set_path = 'data/v1_1/train_set_v1_1_more_than_{0}.txt'.format(TWEETS_NUM_THRESHOLD)
idx_file_path = 'data/v1_1/v1_idx_more_than_{0}.pickle'.format(TWEETS_NUM_THRESHOLD)

save_to_file(dev_set_dict, dev_set_path)
save_to_file(train_set_dict, train_set_path)


# Build IDX and Save
idx = {}
for i, id in enumerate(newTrainDict):
    idx[id] = i
print(len(idx))

with open(idx_file_path, 'wb') as handle:
    pickle.dump(idx, handle, protocol=pickle.HIGHEST_PROTOCOL)

11135
95451
1132


In [70]:
print(train_dict[4032])

['@handle Hahaha!', 'You look like Dave Vanian now? =D', "Oh we renamed Zero, Seamus O'Malley - he looks like he should have a newsboy cap on & a pint of Guinness by his side...", 'Dang, I keep missing all of the #bpal contests - I need a pick me up - this week already = fail.', "I'm hungry! Just sayin'!", 'My cousin said, "Sea Mus" - I know that\'s not unusual to think that but it made me chuckle...', 'And Elena is still in the hospital too, obviously not as bad off as my dad, but my mom told me she & dad passed each other getting CT scans.', 'Most #atheists know the truth about Mother Teresa anyway...', '& esp. if they live in poverty, fuck the parents & child b/c the child ONLY mattered when it was a mass of cells in a womb. Wow - so smart.', 'Just saw it! Thank you, Tired & not in any mood to "talk" - at least I\'m feeling more human today...*hugs*', 'BTW, J ?s his lil knitted monster, Toby, Sascha will bat it to the floor & J will go get it & put it back up on the clock.', "I hope

Split Dev Train
!! There are Some ID have 0 train instance

In [5]:
import random
import numpy as np

dev_split = 0.1
train_split = 1 - dev_split

dev_set_dict = {}
train_set_dict = {}

for id in train_dict:
    target_list = train_dict[id]
    length = len(target_list)
    random.shuffle(target_list)
    split = int(np.ceil(length*dev_split))
    dev_set_dict[id] = target_list[:split]
    train_set_dict[id] = target_list[split:length]
#     print(len(dev_set_dict[id]), len(train_set_dict[id]), length)

print(len(dev_set_dict), len(train_set_dict), len(train_dict))
    
    


9295 9295 9295


Build ID idx and train/dev set save to file

In [7]:
import pickle
import pandas as pd

def save_to_file(target_dict, file_path):
    id_list = []
    sentence_list = []
    for id in target_dict:
        for sentence in target_dict[id]:
            id_list.append(id)
            sentence_list.append(sentence)
      
    id_list = np.array(id_list)
    sentence_list = np.array(sentence_list)
    random_index = np.array(range(len(sentence_list)))
    random.shuffle(random_index)
    id_list = id_list[random_index]
    sentence_list = sentence_list[random_index]
    
    dataframe = pd.DataFrame({'id':id_list,'sentence':sentence_list})
    dataframe.to_csv(file_path,index=False,sep='\t',header=None)
    print(len(id_list))
    return

dev_set_path = 'data/v1/dev_set_v1.txt'
train_set_path = 'data/v1/train_set_v1.txt'
idx_file_path = 'data/v1/v1_idx.pickle'

save_to_file(dev_set_dict, dev_set_path)
save_to_file(train_set_dict, train_set_path)


# Build IDX and Save
idx = {}
for i, id in enumerate(train_dict):
    idx[id] = i
print(len(idx))

with open(idx_file_path, 'wb') as handle:
    pickle.dump(idx, handle, protocol=pickle.HIGHEST_PROTOCOL)


36116
291758
9295


In [9]:
test_file_path = "data/test_tweets_unlabeled.txt"
test_output_path = "data/test_tweets_unlabeled_dataframe.txt"

test_list = []
with open(test_file_path, encoding='utf-8') as tsvfile:
    reader = tsvfile.readlines()
    for i, row in enumerate(reader):
        row = row.strip()
        row, _ =  pre_process(row, max_length)
        test_list.append(row)
#with open(test_output_path, 'w') as output_file:
#    output_file.writelines(test_list)

test_list = np.array(test_list)
dataframe = pd.DataFrame({'sentence':test_list})
dataframe.to_csv(test_output_path,index=False,header=None)
print("Total rows: %d" % len(test_list))

Total rows: 35437


In [10]:
test_file = pd.read_csv(test_output_path, header=None)
test_file

,0
0,Some people say that rappers don’t have feelin...
1,Do you know how to tweet on a Blackberry 8830?...
2,"""Yoga is the cessation of mind."" -Patanjali"
3,"Well, with my millions of dollars, a few bucks..."
4,Cambria hotels free guide cheap discount hotel...
5,May the force of Jesus be with you
6,YEAH! It's finally Monday!
7,Martin Laird won in Las Vegas last week with a...
8,Joe's Crab Shack Fundraiser benefitting the Sa...
9,"i hate my self-portrait painting, I don't know..."
